In [14]:
import functools
import itertools

import numpy as np

from pycobyla.cobyla import Cobyla


def gaussian(x, mu=None, sig=None, A=1):
    n = len(x)
    mu =  np.zeros(n) if mu is None else mu
    sig = np.ones(n) if sig is None else sig

    zz = ((((x - mu) / sig) ** 2) / 2).sum()
    return A * np.exp(-zz)


def neg_gaussian(x, mu=None, sig=None, A=1):
    return -gaussian(x, mu=mu, sig=sig, A=A)

In [15]:
def optimization_out_of_the_boundary_circle(inner_radius=1, outer_radius=2, start_x=(0,0)):
    assert outer_radius > inner_radius
    
    sig = np.random.choice((-1, 1))
    phi = np.random.uniform(low=0, high=2*np.pi*outer_radius) / outer_radius
    x_circle = outer_radius * np.cos(phi)
    y_circle = outer_radius * np.sin(phi)
    
    mu = np.array((x_circle, y_circle))
    
    G = functools.partial(neg_gaussian, mu=mu)
    c1 = lambda x: inner_radius - sum(x ** 2)

    C = (c1,)

    opt = Cobyla(start_x, G, C, rhobeg=0.5, rhoend=1e-15)
    opt.run()
    
    return mu, opt

In [16]:
%matplotlib widget
import matplotlib.pyplot as plt

start_x = np.random.uniform(low=-1, high=1, size=2)
inner_radius = 1
outer_radius = 2
mu, opt = optimization_out_of_the_boundary_circle(inner_radius=inner_radius, outer_radius=outer_radius, start_x=start_x)

plt.close('all')
fig, ax = plt.subplots(figsize=(6, 6))
ax.axis('equal')

inner_circle = plt.Circle((0, 0), inner_radius, color='mistyrose', fill=True)  # boundary
outter_circle = plt.Circle((0, 0), outer_radius, color='lightgrey', fill=False)
ax.add_patch(inner_circle)
ax.add_patch(outter_circle)

ax.plot(*start_x, marker='o', color='blue')
ax.plot(*opt.x, marker='o', color='orange')
ax.plot(*mu, marker='*', color='limegreen')
#ax.plot((start_x[0], opt.x[0]), (start_x[1], opt.x[1]), linestyle='-', color='red')
ax.plot((opt.x[0], mu[0]), (opt.x[1], mu[1]), linestyle='-.', color='red')

uv = (mu - opt.x)
uv /= np.linalg.norm(uv)
vv = (mu[1], -mu[0])
vv /= np.linalg.norm(vv)

assert abs(uv @ vv) < 1e-9


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [17]:
%matplotlib widget
import matplotlib.pyplot as plt

inner_radius = 1
outter_radius = 2
start_x = np.zeros(2)

n_targets = 10
data = tuple(optimization_out_of_the_boundary_circle(inner_radius=inner_radius, outer_radius=outter_radius, start_x=start_x) 
             for _ in range(n_targets))

v_mu = np.array(tuple(mu for mu, opt in data))
v_opt = np.array(tuple(opt.x for mu, opt in data))

plt.close('all')
fig, ax = plt.subplots(figsize=(6, 6))
ax.axis('equal')

inner_circle = plt.Circle((0, 0), inner_radius, color='mistyrose', fill=True)  # boundary
outer_circle = plt.Circle((0, 0), outter_radius, color='lightgrey', fill=False)
ax.add_patch(inner_circle)
ax.add_patch(outer_circle)

ax.plot(*start_x, marker='o', color='blue')
ax.plot(v_opt[:, 0], v_opt[:, 1], linestyle='', marker='o', color='orange')
ax.plot(v_mu[:, 0], v_mu[:, 1],  linestyle='', marker='*', color='limegreen')
for mu, opt_x in zip(v_mu, v_opt):
    #ax.plot((start_x[0], opt_x[0]), (start_x[1], opt_x[1]), linestyle='-', color='red')
    ax.plot((opt_x[0], mu[0]), (opt_x[1], mu[1]), linestyle='-.', color='red')


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [18]:
def optimization_out_of_the_boundary_square(side=1, outer_radius=2, start_x=(0,0)):
    phi = np.random.uniform(low=0, high=2*np.pi*outer_radius) / outer_radius
    x_circle = outer_radius * np.cos(phi)
    y_circle = outer_radius * np.sin(phi)
    
    mu = np.array((x_circle, y_circle))
    
    G = functools.partial(neg_gaussian, mu=mu)
    c1 = lambda x: side/2 - x[0]
    c2 = lambda x: side/2 + x[0]
    c3 = lambda x: side/2 - x[1]
    c4 = lambda x: side/2 + x[1]

    C = (c1, c2, c3, c4)

    opt = Cobyla(start_x, G, C, rhoend=1e-12)
    opt.run()
    
    return mu, opt

In [19]:
%matplotlib widget
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle

side = 2
outer_radius=2
start_x = np.random.uniform(low=-1.5, high=1.5, size=2)

mu, opt = optimization_out_of_the_boundary_square(side=side, outer_radius=outer_radius, start_x=start_x)

plt.close('all')
fig, ax = plt.subplots(figsize=(6, 6))
ax.axis('equal')

boundary = plt.Rectangle((-side/2, -side/2), side, side, color='mistyrose', fill=True)
outer_circle = plt.Circle((0, 0), outter_radius, color='lightgrey', fill=False)
ax.add_patch(boundary)
ax.add_patch(outer_circle)


ax.plot(*start_x, marker='o', color='blue')
ax.plot(*opt.x, marker='o', color='orange')
ax.plot(*mu, marker='*', color='limegreen')
#ax.plot((start_x[0], opt.x[0]), (start_x[1], opt.x[1]), linestyle='-', color='red')
ax.plot((opt.x[0], mu[0]), (opt.x[1], mu[1]), linestyle='-.', color='red')

uv = (mu - opt.x)
leftright = np.array((1,0)) @ uv
updown = np.array((0,1)) @ uv

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [20]:
%matplotlib widget
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle

side = 2
outer_radius=2
start_x = np.zeros(2)

n_targets = 10
data = tuple(optimization_out_of_the_boundary_square(side=side, outer_radius=outer_radius, start_x=start_x) 
             for _ in range(n_targets))
v_mu = np.array(tuple(mu for mu, opt in data))
v_opt = np.array(tuple(opt.x for mu, opt in data))

plt.close('all')
fig, ax = plt.subplots(figsize=(6, 6))
ax.axis('equal')

boundary = plt.Rectangle((-side/2, -side/2), side, side, color='mistyrose', fill=True)
outer_circle = plt.Circle((0, 0), outter_radius, color='lightgrey', fill=False)
ax.add_patch(boundary)
ax.add_patch(outer_circle)

ax.plot(*start_x, marker='o', color='blue')
ax.plot(v_opt[:, 0], v_opt[:, 1], linestyle='', marker='o', color='orange')
ax.plot(v_mu[:, 0], v_mu[:, 1],  linestyle='', marker='*', color='limegreen')

for mu, opt_x in zip(v_mu, v_opt):
    #ax.plot((start_x[0], opt_x[0]), (start_x[1], opt_x[1]), linestyle='-', color='red')
    ax.plot((opt_x[0], mu[0]), (opt_x[1], mu[1]), linestyle='-.', color='red')


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …